In [38]:
!pip install kaggle

In [39]:
import os
import json
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM # Dense-fully connected layer, embedding-strating layer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [40]:
kaggle_dictionary=json.load(open('kaggle.json'))

In [41]:
kaggle_dictionary.keys()

dict_keys(['username', 'key'])

In [42]:
os.environ['USERNAME']=kaggle_dictionary['username']
os.environ['KEY']=kaggle_dictionary['key']

In [43]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews


Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
imdb-dataset-of-50k-movie-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [44]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [45]:
# let's unzip the dataset

with ZipFile('imdb-dataset-of-50k-movie-reviews.zip','r') as zip_ref:
  zip_ref.extractall()

In [46]:
!ls

'IMDB Dataset.csv'   imdb-dataset-of-50k-movie-reviews.zip   kaggle.json   sample_data


In [47]:
import pandas as pd

In [48]:
df=pd.read_csv('/content/IMDB Dataset.csv')

In [49]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [50]:
df.isnull().sum()

,0
review,0
sentiment,0


In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [52]:
df['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [53]:
df.duplicated().sum()

418

In [54]:
df.drop_duplicates(inplace=True)

In [55]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [56]:
df['sentiment'].value_counts()

,count
sentiment,
positive,24884
negative,24698


In [57]:
# from sklearn.preprocessing import LabelEncoder

# label=LabelEncoder()
# df['sentiment']=df['sentiment'].apply(label)

In [58]:
df.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)

<ipython-input-58-fb62f5dd8c08>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace({"sentiment":{"positive":1,"negative":0}},inplace=True)


In [59]:

df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [60]:
from sklearn.model_selection import train_test_split

train_data,test_data=train_test_split(df,test_size=0.2,random_state=42)

In [61]:
# DATA PREPROCESSING

In [62]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re
from nltk.stem.porter import PorterStemmer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
# def text_preprocessor(text):
  # text=text.lower()
  # text=re.sub('[^a-zA-Z\s]','',text)

  # y=[]
  # for word in text.split(' '):
  #   if not word in stopwords.words('english'):
  #     y.append(word)


  # ps=PorterStemmer()
  # y=[ps.stem(word) for word in y]

  # return " ".join(y)

In [64]:
# df['review']=df['review'].apply(text_preprocessor)

In [65]:
# text_preprocessor("A wonderful little production. <br /><br />The.")

In [66]:
tokenizer=Tokenizer(num_words=5000)
tokenizer.fit_on_texts(train_data['review'])
X_train=pad_sequences(tokenizer.texts_to_sequences(train_data['review']),maxlen=200)
X_test=pad_sequences(tokenizer.texts_to_sequences(test_data['review']),maxlen=200)

In [67]:
X_train

array([[   0,    0,    0, ...,  352,  202,  134],
       [   0,    0,    0, ..., 4247,    2, 2721],
       [ 138,   14, 2503, ...,  414,   25,  195],
       ...,
       [2277,   93,   33, ...,    3,  454,  156],
       [   0,    0,    0, ...,  244,  103,  126],
       [   0,    0,    0, ...,    4,    1, 2902]], dtype=int32)

In [68]:
y_train=train_data['sentiment']
y_test=test_data['sentiment']

In [69]:
y_train

,sentiment
7837,0
4814,0
35458,1
3446,0
24478,0
...,...
11304,0
45059,0
38405,0
860,1


**Building the LSTM model**

In [77]:
model=Sequential()
model.add(Embedding(input_dim=5000, output_dim=128, input_shape=(200,)))
model.add(LSTM(128, dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [78]:
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ (None, 200, 128)            │         640,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 128)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 771,713 (2.94 MB)

 Trainable params: 771,713 (2.94 MB)

 Non-trainable params: 0 (0.00 B)

In [79]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [80]:
model.fit(X_train,y_train,epochs=5,batch_size=64,validation_split=0.2)

Epoch 1/5
496/496 ━━━━━━━━━━━━━━━━━━━━ 402s 799ms/step - accuracy: 0.7226 - loss: 0.5337 - val_accuracy: 0.8558 - val_loss: 0.3561
Epoch 2/5
496/496 ━━━━━━━━━━━━━━━━━━━━ 407s 730ms/step - accuracy: 0.8353 - loss: 0.3810 - val_accuracy: 0.8492 - val_loss: 0.3588
Epoch 3/5
496/496 ━━━━━━━━━━━━━━━━━━━━ 360s 727ms/step - accuracy: 0.8017 - loss: 0.4343 - val_accuracy: 0.8428 - val_loss: 0.3747
Epoch 4/5
496/496 ━━━━━━━━━━━━━━━━━━━━ 376s 715ms/step - accuracy: 0.8741 - loss: 0.3080 - val_accuracy: 0.8635 - val_loss: 0.3309
Epoch 5/5
496/496 ━━━━━━━━━━━━━━━━━━━━ 362s 729ms/step - accuracy: 0.9018 - loss: 0.2543 - val_accuracy: 0.8708 - val_loss: 0.3204


In [82]:
loss,accuracy=model.evaluate(X_test,y_test)
print(f"Test loss :{loss}")
print(f"Test accuracy :{accuracy}")

310/310 ━━━━━━━━━━━━━━━━━━━━ 31s 100ms/step - accuracy: 0.8753 - loss: 0.3046
Test loss :0.3110719621181488
Test accuracy :0.8729454278945923


**Building out our Predictive System:**





In [98]:
def predict_sentiment(review):
  sequence=tokenizer.texts_to_sequences([review])
  padded_seq=pad_sequences(sequence,maxlen=200)
  prediction=model.predict(padded_seq)
  print(prediction)
  sentiment='positive' if prediction[0][0]>0.5 else 'negative'
  return sentiment

In [99]:
predict_sentiment("This is good movie")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
[[0.47393867]]


'negative'

In [102]:
print(predict_sentiment("This movie was absolutely fantastic!"))
print(predict_sentiment("Worst movie ever, total waste of time."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step
[[0.59520334]]
positive
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
[[0.00514475]]
negative


In [103]:
print(predict_sentiment("the movie was ok, but not that good."))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step
[[0.1020067]]
negative
